### 1. Definição do problema
> O objetivo deste trabalho é apresentar uma análise sobre os preços de produtos importados pelo Brasil, comparando os valores declarados numa operação de importação com os preços praticados no comércio internacional.
Para isso, primeiramente, será realizado um levantamento de preços junto ao mercado mundial para um grupo de mercadorias de alto valor agregado.
Em seguida, buscar-se-á os preços desses mesmos produtos declarados em processos de importação de mercadorias.
Por fim, far-se-á a comparação entre as médias desses valores, visando a uma conclusão quanto a indícios de fraudes - subfaturamento ou superfaturamento - nas operações de importação realizadas pelas empresas brasileiras.

### 2. A coleta de dados
> Os datasets usados neste trabalho foram coletados de fontes diversas. 
> O DataCoSupplyChainDataset tem como fonte o site __www.kaggle.com__ 

__Bold por favor__ <br>
_Italico Olha isso_ <br>
$ a + bX$ <br>
__________ dfjlkaj dlkjf lkajkdj
> item 
>>item2
>>>item3
- item 1
- item 2
1. item 1
2. item 2 <br>
<font color=blue>"font color=blue"</font><br>
paragrafog <br> outro paragrafo<br>
#isto é comentário

In [15]:
from bs4 import BeautifulSoup
import requests

#bibliotecas para requisição de páginas dinâmicas
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
import time 

#Preparando a aquisição das páginas
url = "https://www.alibaba.com/trade/search?IndexArea=product_en&CatId=5904002&fsb=y&viewtype=&tab=&SearchScene=&SearchText=phone+display+lcd+screen"

driver = webdriver.Firefox('/media/joao/HDLinux/tools')
driver.get(url)
time.sleep(3)

req = driver.page_source
soup = BeautifulSoup(req, "html.parser")

#Aqui é para páginas estáticas. Não usa o webdriver
#req = requests.get(url)
#soup = BeautifulSoup(req.text, "html.parser")



#Adquirindo a lista dos produtos
classes = soup.find('div', class_ = "organic-list app-organic-search__list")

#Agora vamos buscar cada os itens da lista
items =  classes.find_all(class_ = 'organic-gallery-offer-outter J-offer-wrapper')

#Para cada item, pegar a descrição () e o preco
desc_items = {} #tr selecionados
for item in items:
    title = item.find('h4').get('title')
    price = item.find('p', class_ = 'elements-offer-price-normal').get('title')
    desc_items[title] = price 

print(len(desc_items))

print(desc_items)      


48
{'Phone Lcd Iphone Screen Display Cell Phone LCD For IPhone XS OLED Touch Screen Display Replacement': '$29.70-$42.20', 'Phone Lcd Screen Display Wholesale Mobile Phone Parts Lcd Touch Screen Lcd Display Digitizer Replacement': '$50.00-$52.50', 'Phone Lcd Screen Display Mobile Phone Lcd Display Supplier Mobile Phone Lcd Screen Display For Samsung Galaxy J7 Prime Display Lcd Panel': '$12.81-$13.97', 'Phones Touch Screen For Samsung J6 Plus SM-J610FN/DS J610G SM-J610G/DS Mobile Phone LCDs Screen Touch LCD Display': '$12.36-$17.02', 'Lcd Display Zkong 2.7 Inch Esl Lcd Display Factory Supermarket Electronic Lcd Price Tag': '$7.30-$10.80', 'Lcd Screen Lcd Laser Repair Machine FORWARD Hot Selling LCD Blue Light Laser Screen Separating Machine For All Flat Screen Separating And Repairing': '$1,278.96', 'Gx gx3 Hx Jk Rj Amoled Oled Mobile Phone Lcds Screen Replacement With Touch Screen Digitizer Assembly For Iphone5 6 7 8 10 X xr': '$9.64-$110.98', 'lcd screen wholesale for samsung lcd scre

In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib as plt
import seaborn as sns

In [10]:
df = pd.read_csv('Datasets/DataCoSupplyChainDataset.csv', encoding='unicode_escape')

In [22]:
df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [23]:
df.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

In [24]:
df.shape


(180519, 53)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [26]:
df.columns[df.isnull().any()]

Index(['Customer Lname', 'Customer Zipcode', 'Order Zipcode',
       'Product Description'],
      dtype='object')

In [27]:
#Excluindo colunas desnecessárias. "inplace=True" apaga do dataframe de trabalho.
df.drop(columns=['Customer Lname'], inplace=True)

In [18]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 52 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [25]:
import requests
import csv
from bs4 import BeautifulSoup


f = csv.writer(open('z-artist-names.csv', 'w'))
f.writerow(['Name', 'Link'])

pages = []

for i in range(1, 5):
    url = 'https://web.archive.org/web/20121007172955/https://www.nga.gov/collection/anZ' + str(i) + '.htm'
    pages.append(url)


for item in pages:
    page = requests.get(item)
    soup = BeautifulSoup(page.text, 'html.parser')

    last_links = soup.find(class_='AlphaNav')
    last_links.decompose()

    artist_name_list = soup.find(class_='BodyText')
    print(artist_name_list)
    artist_name_list_items = artist_name_list.find_all('a')

    for artist_name in artist_name_list_items:
        names = artist_name.contents[0]
        links = 'https://web.archive.org' + artist_name.get('href')

        f.writerow([names, links])



<div class="BodyText">
<!-- InstanceBeginEditable name="BodyText" -->
<h3>Artist names beginning with Z</h3><table>
<tr valign="top"><td><a href="/web/20121007172955/https://www.nga.gov/cgi-bin/tsearch?artistid=11630">Zabaglia, Niccola</a></td><td>Italian, 1664 - 1750</td></tr>
<tr valign="top"><td><a href="/web/20121007172955/https://www.nga.gov/cgi-bin/tsearch?artistid=34202">Zaccone, Fabian</a></td><td>American, 1910 - 1992</td></tr>
<tr valign="top"><td><a href="/web/20121007172955/https://www.nga.gov/cgi-bin/tsearch?artistid=3475">Zadkine, Ossip</a></td><td>French, 1890 - 1967</td></tr>
<tr valign="top"><td><a href="/web/20121007172955/https://www.nga.gov/cgi-bin/tsearch?artistid=25135">Zaech, Bernhard</a></td><td>German, active c. 1650</td></tr>
<tr valign="top"><td><a href="/web/20121007172955/https://www.nga.gov/cgi-bin/tsearch?artistid=2298">Zagar, Jacob</a></td><td>Flemish, c. 1530 - after 1580</td></tr>
<tr valign="top"><td><a href="/web/20121007172955/https://www.nga.gov/cg